In [1]:
__file__=''
import os,sys
import pandas as pd
sys.path.append(os.path.join(os.path.dirname(__file__),'../LIB/'))
sys.path.append(os.path.join(os.path.dirname(__file__),'../../../../automl/automl_libs/'))
from env import FILE
import utils
from itertools import combinations
from feature_engineering import *
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
import gc
from sklearn.metrics import log_loss
import pickle

In [2]:
train = pd.read_pickle(FILE.train_appends.value)
# train = pd.read_pickle(FILE.train_agg_v3_select.value)
print('train shape is: {}'.format(train.shape))
test = pd.read_pickle(FILE.test_appends.value)
# test = pd.read_pickle(FILE.test_agg_v3_select.value)
print('test shape is: {}'.format(test.shape))

# train = train1.iloc[:,:200].copy()
# train['click'] = train1['click'].copy()
# test = test1.iloc[:,:200].copy()

train shape is: (1001650, 43)
test shape is: (40024, 42)


# LGBM

In [3]:
from sklearn.model_selection import train_test_split
print_to_file = False 
test_run = False 

def get_time(timezone='America/New_York', time_format='%Y-%m-%d %H:%M:%S'):
    from datetime import datetime
    from dateutil import tz

    # METHOD 1: Hardcode zones:
    from_zone = tz.gettz('UTC')
    to_zone = tz.gettz(timezone)

    utc = datetime.utcnow()

    # Tell the datetime object that it's in UTC time zone since 
    # datetime objects are 'naive' by default
    utc = utc.replace(tzinfo=from_zone)

    # Convert time zone
    est = utc.astimezone(to_zone)

    return est.strftime(time_format)

import sys, time
class Logger(object):
    def __init__(self, logtofile=True, logfilename='log'):
        self.terminal = sys.stdout
        self.logfile = "{}_{}.log".format(logfilename, int(time.time()))
        self.logtofile = logtofile

    def write(self, message):
        #         self.terminal.write(message)
        if self.logtofile:
            self.log = open(self.logfile, "a")
            self.log.write('[' + get_time() + '] ' + message)
            self.log.close()

    def flush(self):
        # this flush method is needed for python 3 compatibility.
        # this handles the flush command by doing nothing.
        # you might want to specify some extra behavior here.
        pass


def divert_printout_to_file():
    sys.stdout = Logger(logfilename='logfile')

if print_to_file:
    divert_printout_to_file()  # note: comment this to use pdb

import numpy as np
import pandas as pd
import gc
import time
from contextlib import contextmanager
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

@contextmanager
def timer(title):
    t0 = time.time()
    yield
    print("{} - done in {:.0f}s".format(title, time.time() - t0))

# LightGBM GBDT with KFold or Stratified KFold
# Parameters from Tilii kernel: https://www.kaggle.com/tilii7/olivier-lightgbm-parameters-by-bayesian-opt/code
def kfold_lightgbm(df, train_df, test_df, holdout, num_folds, submission_file_name, fe_img_name, stratified = False, debug= False, colsample=0.67, max_depth=8, num_leaves=31, min_child_samples=20, subsample=0.7, reg_lambda=0.3, lr=0.04, seed=1001, verbose=100, rounds=None, target='click'):
    print(train_df.shape, test_df.shape, holdout.shape)
    print('MEAN: train({}) vs holdout({}): '.format(len(train_df), len(holdout)), train_df[target].mean(), holdout[target].mean())
    # Divide in training/validation and test data
    if df is not None:
        train_df = df[df[target].notnull()]
        test_df = df[df[target].isnull()]
        print("Starting LightGBM. Train shape: {}, test shape: {}".format(train_df.shape, test_df.shape))
        del df
        gc.collect()
    # Cross validation model
    if stratified:
        folds = StratifiedKFold(n_splits= num_folds, shuffle=True, random_state=seed)
    else:
        folds = KFold(n_splits= num_folds, shuffle=True, random_state=seed)
        
    # Create arrays and dataframes to store results
    oof_preds = np.zeros(train_df.shape[0])
    sub_preds = np.zeros(test_df.shape[0])
    holdout_final_preds = np.zeros(holdout.shape[0])
    feature_importance_df = pd.DataFrame()
    feature_importance_gain_df = pd.DataFrame()
    feats = [f for f in train_df.columns if f not in [target,'time','instance_id','index']]
    train_scores = []
    holdout_scores = []
    scores = []
    diff_val_holdout = []
    
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], train_df[target])):
#         print('valid index : ',list(valid_idx)[:5])
        train_x, train_y = train_df[feats].iloc[train_idx], train_df[target].iloc[train_idx]
        valid_x, valid_y = train_df[feats].iloc[valid_idx], train_df[target].iloc[valid_idx]
#         print('MEAN: train({}) vs valid({}): '.format(len(train_y), len(valid_y)), np.mean(train_y), np.mean(valid_y))

        clf = LGBMClassifier(
            nthread=18,
            n_estimators=30000,
            learning_rate=lr,
#             max_bin =10000,
#             num_leaves=num_leaves,
#             colsample_bytree=colsample, # 0.67
#             subsample=subsample,
#             subsample_freq=0, ## disable subsampling
#             max_depth=max_depth,
#             reg_alpha=0.65,
#             reg_lambda=reg_lambda,
#             min_split_gain=0.0222415,
#             min_child_weight=39.3259775,
#             min_child_samples=min_child_samples,
            silent=-1,
            verbose=-1, )
        if rounds is not None:
            clf.n_estimators = rounds
            clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], 
                eval_metric= 'logloss', verbose=verbose)
            oof_preds[valid_idx] = clf.predict_proba(valid_x)[:, 1]
            sub_preds += clf.predict_proba(test_df[feats])[:, 1] / folds.n_splits
            holdout_preds = clf.predict_proba(holdout[feats])[:, 1] 
        else:
            clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], 
                eval_metric= 'logloss', verbose=verbose, early_stopping_rounds= 200)
            oof_preds[valid_idx] = clf.predict_proba(valid_x, num_iteration=clf.best_iteration_)[:, 1]
            sub_preds += clf.predict_proba(test_df[feats], num_iteration=clf.best_iteration_)[:, 1] / folds.n_splits
            holdout_preds = clf.predict_proba(holdout[feats], num_iteration=clf.best_iteration_)[:, 1] 
            
        holdout_final_preds += holdout_preds / folds.n_splits
        score = log_loss(valid_y, oof_preds[valid_idx])
        train_score = clf.best_score_['training']['binary_logloss']
        holdout_score = log_loss(holdout[target], holdout_preds)
        diff = abs(score - holdout_score)
        best_rounds = rounds if rounds is not None else clf.best_iteration_
        print('Fold %2d [%5d] AUC : ho: %.6f / te: %.6f / tr: %.6f (diff: %.6f)' % (n_fold + 1, best_rounds, holdout_score, score,  train_score, diff))
        scores.append(score)
        train_scores.append(train_score)
        holdout_scores.append(holdout_score)
        diff_val_holdout.append(diff)
        
        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = feats
        fold_importance_df["importance"] = clf.feature_importances_
        fold_importance_df["fold"] = n_fold + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        
        fold_importance_gain_df = pd.DataFrame()
        fold_importance_gain_df["feature"] = feats
        fold_importance_gain_df["importance"] = clf.booster_.feature_importance(importance_type='gain')
        fold_importance_gain_df["fold"] = n_fold + 1
        feature_importance_gain_df = pd.concat([feature_importance_gain_df, fold_importance_gain_df], axis=0)
        
        del clf, train_x, train_y, valid_x, valid_y
        gc.collect()
    holdout_roc = log_loss(holdout[target], holdout_final_preds)
    holdout_mean = np.mean(holdout_scores)
    full_te_mean = np.mean(scores)
    full_tr_mean = np.mean(train_scores)
#     print('Full AUC score %.6f' % roc_auc_score(train_df['TARGET'], oof_preds))
    print('Full HO score %.6f' % holdout_roc)
    print('FULL HO mean {:.6f}, std {:.6f}'.format(holdout_mean, np.std(holdout_scores)))
    print('FULL TE mean {:.6f}, std {:.6f}'.format(full_te_mean, np.std(scores)))
    print('FULL TR mean {:.6f}, std {:.6f}'.format(full_tr_mean, np.std(train_scores)))
    print('FULL DIFF mean {:.6f}, std {:.6f}'.format(np.mean(diff_val_holdout), np.std(diff_val_holdout)))
    # Write submission file and plot feature importance
    if not debug:
        print('saving...')
        test_df['predicted_score'] = sub_preds
        test_df[['instance_id', 'predicted_score']].to_csv(submission_file_name, index= False)
#     if not print_to_file:
#         display_importances(feature_importance_df, fe_img_name)
    feature_importance_df = feature_importance_df[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False).reset_index()
    feature_importance_gain_df = feature_importance_gain_df[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False).reset_index()
    return feature_importance_df, feature_importance_gain_df,holdout_roc,holdout_mean,full_te_mean,full_tr_mean,oof_preds

# Display/plot feature importance
def display_importances(feature_importance_df_, fe_img_name):
    cols = feature_importance_df_[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)[:40].index
    best_features = feature_importance_df_.loc[feature_importance_df_.feature.isin(cols)]
    plt.figure(figsize=(8, 10))
    sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False))
    plt.title('LightGBM Features (avg over folds)')
    plt.tight_layout()
    plt.savefig(fe_img_name+'.png')


def convert_and_save_imp_df(fe_imp_df, dumpfilename):
    fe_imp_df_mean = fe_imp_df[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False).reset_index()
    pickle.dump(fe_imp_df_mean, open(dumpfilename,'wb'))
    
    

def select_holdout(train,test,shift=1,seed=6):
    samples_size = int(len(test) )
    assert shift in range(1,8)
    test_time_min = test.time.min()
    test_time_max = test.time.max()
    train_time_min = train.time.min()
    train_time_max = train.time.max()
    window_lower_bound = test_time_min - shift*60*60*24
    window_upper_bound = test_time_max - shift*60*60*24
    available_train = train[(train.time<=window_upper_bound) & (train.time>=window_lower_bound)]
    holdout = available_train.sample(n=samples_size,random_state=seed).copy()
    train_split_index = list(set(train.index) - set(holdout.index))
    train_split = train.iloc[train_split_index].copy()
    return train_split,holdout

In [4]:
def runlgb(train, test, holdout):
    colsamples = [0.07]#[0.1,0.15,0.2]#[0.03,0.04,0.05,0.06,0.07,0.08]
    seeds = [20]#[300,4000,50000,600000,7000000,80000000,523445,31275479] # 20
    depth = [5]
    leaves = [16]
    min_child_sam = [20]#, 800]
    subsamples = [1]#0.8, 0.7, 0.6, 0.5, 0.4] # was 1
    reg_lambdas = [0.5]
    # lrs = lrs.tolist()
    lrs2 = [0.05]
    nfolds = 5
    rounds = [None] #[1000]#, 1300, 1600, 1900, 2200, 2500]
    for seed in seeds:
        for colsample in colsamples:
            for d in depth:
                for l in leaves:
                    for mcs in min_child_sam:
                        for subsample in subsamples:
                            for reg_lambda in reg_lambdas:
                                for lr in lrs2:
                                    for r in rounds:
                                        filename = 'fe_936_col{}_lr{}_n{}'.format(len(train.columns), lr, nfolds)
                                        print('#############################################')
                                        print(colsample, seed, d, l, mcs, subsample, reg_lambda, lr, 'nfolds:', nfolds)
                                        print('#############################################')
                                        numfeats = len(train.columns)
                                        with timer("Run LightGBM with kfold"):
                                            return kfold_lightgbm(None, train, test, holdout, nfolds, filename+'.csv', filename, colsample=colsample, verbose=None, max_depth=d, num_leaves=l, min_child_samples=mcs, subsample=subsample, reg_lambda=reg_lambda, lr=lr, seed=seed, stratified=True, rounds=r,debug=True)                                        

In [5]:
def drop_loop_each(train,test,file_name='drop_each_col_score_0915.csv',ignore_list = ['click','instance_id','time']):
    try:
        df_drop_result = pd.read_csv(file_name)
        processed_list = list(df_drop_result['colName'].values)
    except:
        df_drop_result = pd.DataFrame()
        processed_list = []
    for col in ['nodrop']+list(train.columns):
        
        if col in processed_list:
            continue
        if col not in ignore_list:
            print('processing {}, {}/{}'.format(col,len(processed_list),len(train.columns)+1))
            if col == 'nodrop':
                train_drop = train.copy()
                test_drop = test.copy()
            else:
                train_drop = train.drop(col,axis=1).copy()
                test_drop = test.drop(col,axis=1).copy()
            train_df,holdout = select_holdout(train_drop,test_drop,shift=1,seed=41)
            print('train shape: {}, holdout shape: {}'.format(train_df.shape,holdout.shape))
            print('train mean: {}, holdout mean: {}'.format(train_df.click.mean(),holdout.click.mean()))
            feature_importance_df, feature_importance_gain_df,holdout_roc,holdout_mean,full_te_mean,full_tr_mean,oof_preds = runlgb(train_df, test, holdout)
            score_new_df = pd.DataFrame({'colName':[col],
                                         'holdout_overall':[holdout_roc],
                                         'holdout_mean':[holdout_mean],
                                         'val_mean':[full_te_mean],
                                         'train_mean':[full_tr_mean]})
            df_drop_result = pd.concat([df_drop_result,score_new_df],ignore_index=True,sort=False)
            df_drop_result.to_csv(file_name,index=False)
            processed_list.append(col)
            
            best = df_drop_result.sort_values(['val_mean','holdout_overall']).iloc[0]
            best_ho_full = best['holdout_overall']
            best_ho_mean = best['holdout_mean']
            best_cv_mean = best['val_mean']
            print('best holdout overall score is: {}'.format(best_ho_full))
            print('best holdout mean score is: {}'.format(best_ho_mean))
            print('best cv mean score is: {}'.format(best_cv_mean))
    return df_drop_result
            
        

In [8]:
file_name_formater = 'drop_each_col_score_V3_4_round{}_appends.csv'
dump_name = 'drop_cols_V3_4_appends.pkl'
try:
    drop_list = pickle.load(dump_name,'rb')
except:
    drop_list = []
r = len(drop_list)+1
train = train.drop(drop_list,axis=1)
test = test.drop(drop_list,axis=1)
while True:
    df_drop_result = drop_loop_each(train,test,file_name_formater.format(r))
    df_drop_result_base = df_drop_result[df_drop_result['colName']=='nodrop'].iloc[0]
    base_ho = df_drop_result_base['holdout_overall']
    base_cv = df_drop_result_base['val_mean']
    df_drop_result_better = df_drop_result[(df_drop_result['holdout_overall']<base_ho) &(df_drop_result['val_mean']<base_cv)].copy()
#     df_drop_result_better = df_drop_result[(df_drop_result['holdout_overall']<base_ho)].copy()
    if len(df_drop_result_better) == 0:
        break
        print('finish')
    else:
        df_drop_result_better = df_drop_result_better.sort_values(['val_mean','holdout_overall'])
        col = df_drop_result_better.iloc[0]['colName']
        train = train.drop(col,axis=1)
        test = test.drop(col,axis=1)
        drop_list.append(col)
        pickle.dump(drop_list,open(dump_name,'wb'))
        r+=1
        print('current drop list is:{}'.format(drop_list))

processing carrier, 4/44
train shape: (961626, 42), holdout shape: (40024, 42)
train mean: 0.19797613625255556, holdout mean: 0.21007395562662404
#############################################
0.07 20 5 16 20 1 0.5 0.05 nfolds: 5
#############################################
(961626, 42) (40024, 42) (40024, 42)
MEAN: train(961626) vs holdout(40024):  0.19797613625255556 0.21007395562662404
Fold  1 [  828] AUC : ho: 0.426854 / te: 0.416257 / tr: 0.404292 (diff: 0.010597)
Fold  2 [  732] AUC : ho: 0.426929 / te: 0.415463 / tr: 0.405732 (diff: 0.011466)
Fold  3 [  657] AUC : ho: 0.426954 / te: 0.416464 / tr: 0.406459 (diff: 0.010490)
Fold  4 [  909] AUC : ho: 0.426864 / te: 0.417004 / tr: 0.403407 (diff: 0.009860)
Fold  5 [  589] AUC : ho: 0.426764 / te: 0.415958 / tr: 0.407668 (diff: 0.010806)
Full HO score 0.426205
FULL HO mean 0.426873, std 0.000066
FULL TE mean 0.416229, std 0.000513
FULL TR mean 0.405511, std 0.001518
FULL DIFF mean 0.010644, std 0.000518
Run LightGBM with kfold - don

Fold  1 [  637] AUC : ho: 0.426942 / te: 0.416296 / tr: 0.406825 (diff: 0.010645)
Fold  2 [  684] AUC : ho: 0.426759 / te: 0.415439 / tr: 0.406331 (diff: 0.011320)
Fold  3 [  759] AUC : ho: 0.427054 / te: 0.416404 / tr: 0.405162 (diff: 0.010650)
Fold  4 [  787] AUC : ho: 0.426785 / te: 0.416952 / tr: 0.404855 (diff: 0.009832)
Fold  5 [  654] AUC : ho: 0.426994 / te: 0.415959 / tr: 0.406674 (diff: 0.011035)
Full HO score 0.426275
FULL HO mean 0.426907, std 0.000116
FULL TE mean 0.416210, std 0.000501
FULL TR mean 0.405969, std 0.000807
FULL DIFF mean 0.010697, std 0.000501
Run LightGBM with kfold - done in 118s
best holdout overall score is: 0.4263294520505327
best holdout mean score is: 0.426925620084582
best cv mean score is: 0.4161887235176457
processing adid, 12/44
train shape: (961626, 42), holdout shape: (40024, 42)
train mean: 0.19797613625255556, holdout mean: 0.21007395562662404
#############################################
0.07 20 5 16 20 1 0.5 0.05 nfolds: 5
#################

Fold  3 [  600] AUC : ho: 0.426961 / te: 0.416543 / tr: 0.407366 (diff: 0.010418)
Fold  4 [ 1083] AUC : ho: 0.427077 / te: 0.416908 / tr: 0.401254 (diff: 0.010169)
Fold  5 [  533] AUC : ho: 0.426933 / te: 0.415990 / tr: 0.408321 (diff: 0.010943)
Full HO score 0.426303
FULL HO mean 0.426948, std 0.000071
FULL TE mean 0.416286, std 0.000460
FULL TR mean 0.405820, std 0.002435
FULL DIFF mean 0.010661, std 0.000405
Run LightGBM with kfold - done in 120s
best holdout overall score is: 0.4263294520505327
best holdout mean score is: 0.426925620084582
best cv mean score is: 0.4161887235176457
processing f_channel, 19/44
train shape: (961626, 42), holdout shape: (40024, 42)
train mean: 0.19797613625255556, holdout mean: 0.21007395562662404
#############################################
0.07 20 5 16 20 1 0.5 0.05 nfolds: 5
#############################################
(961626, 42) (40024, 42) (40024, 42)
MEAN: train(961626) vs holdout(40024):  0.19797613625255556 0.21007395562662404
Fold  1 [  69

Fold  5 [  654] AUC : ho: 0.426994 / te: 0.415959 / tr: 0.406674 (diff: 0.011035)
Full HO score 0.426275
FULL HO mean 0.426907, std 0.000116
FULL TE mean 0.416210, std 0.000501
FULL TR mean 0.405969, std 0.000807
FULL DIFF mean 0.010697, std 0.000501
Run LightGBM with kfold - done in 148s
best holdout overall score is: 0.4263294520505327
best holdout mean score is: 0.426925620084582
best cv mean score is: 0.4161887235176457
processing creative_is_download, 26/44
train shape: (961626, 42), holdout shape: (40024, 42)
train mean: 0.19797613625255556, holdout mean: 0.21007395562662404
#############################################
0.07 20 5 16 20 1 0.5 0.05 nfolds: 5
#############################################
(961626, 42) (40024, 42) (40024, 42)
MEAN: train(961626) vs holdout(40024):  0.19797613625255556 0.21007395562662404
Fold  1 [  637] AUC : ho: 0.426942 / te: 0.416296 / tr: 0.406825 (diff: 0.010645)
Fold  2 [  684] AUC : ho: 0.426759 / te: 0.415439 / tr: 0.406331 (diff: 0.011320)
Fo

KeyboardInterrupt: 

In [13]:
train.shape

(1001650, 35)

In [14]:
train.to_pickle(FILE.train_agg_v3_select.value)
test.to_pickle(FILE.test_agg_v3_select.value)

In [29]:
train_df,holdout = select_holdout(train,test,shift=1,seed=41)
print('train shape: {}, holdout shape: {}'.format(train_df.shape,holdout.shape))
print('train mean: {}, holdout mean: {}'.format(train_df.click.mean(),holdout.click.mean()))
feature_importance_df, feature_importance_gain_df,holdout_roc,holdout_mean,full_te_mean,full_tr_mean,oof_preds = runlgb(train_df, test, holdout)

train shape: (961626, 26), holdout shape: (40024, 26)
train mean: 0.19797613625255556, holdout mean: 0.21007395562662404
#############################################
0.07 20 5 16 20 1 0.5 0.05 nfolds: 5
#############################################
(961626, 26) (40024, 26) (40024, 26)
MEAN: train(961626) vs holdout(40024):  0.19797613625255556 0.21007395562662404
Fold  1 [  524] AUC : ho: 0.427050 / te: 0.416522 / tr: 0.409707 (diff: 0.010529)
Fold  2 [  564] AUC : ho: 0.426715 / te: 0.415591 / tr: 0.409485 (diff: 0.011124)
Fold  3 [  529] AUC : ho: 0.426823 / te: 0.416415 / tr: 0.409677 (diff: 0.010408)
Fold  4 [  601] AUC : ho: 0.426446 / te: 0.417102 / tr: 0.408739 (diff: 0.009344)
Fold  5 [  611] AUC : ho: 0.427144 / te: 0.416160 / tr: 0.408859 (diff: 0.010985)
Full HO score 0.426396
FULL HO mean 0.426836, std 0.000248
FULL TE mean 0.416358, std 0.000492
FULL TR mean 0.409293, std 0.000413
FULL DIFF mean 0.010478, std 0.000627
Run LightGBM with kfold - done in 98s


In [38]:
df_drop_result[df_drop_result['colName']=='nodrop']

,colName,holdout_overall,holdout_mean,val_mean,train_mean
0,nodrop,0.426396,0.426836,0.416358,0.409293


In [30]:
add_col_result = pd.read_csv('add_each_col_score_0915_round1.csv')

In [33]:
add_col_result[add_col_result['colName'] == 'noadd']

,colName,holdout_overall,holdout_mean,val_mean,train_mean
3,noadd,0.426737,0.427272,0.416447,0.409458


In [37]:
add_col_result.sort_values(['holdout_overall','val_mean'])

,colName,holdout_overall,holdout_mean,val_mean,train_mean
32,user3_ads1_channel0_hour0_count,0.426546,0.427160,0.416446,0.408414
44,user3_ads0_channel0_hour0_count,0.426546,0.427160,0.416446,0.408414
107,user3_ads2_channel0_hour0_count,0.426546,0.427160,0.416446,0.408414
246,user0_ads0_channel0_hour1_reverse_cumulative_c...,0.426563,0.427101,0.416466,0.408773
265,user4_ads4_channel3_hour0_reverse_cumulative_c...,0.426576,0.427151,0.416478,0.407884
122,user4_ads0_channel3_hour0_reverse_cumulative_c...,0.426580,0.427162,0.416495,0.407799
174,user4_ads1_channel3_hour0_reverse_cumulative_c...,0.426580,0.427162,0.416495,0.407799
19,user1_ads0_channel2_hour1_reverse_cumulative_c...,0.426583,0.427108,0.416476,0.409140
165,user4_ads3_channel1_hour0_count,0.426584,0.427131,0.416449,0.409146
48,user4_ads1_channel2_hour0_reverse_cumulative_c...,0.426593,0.427110,0.416491,0.409038
